In [3]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone

# Setup Faker for generating mock data
fake = faker.Faker()

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

In [5]:
# Data stores for unique values
users = {}
user_devices = {}
user_friends = {}
user_game_types = {}
game_types = {}
add_fund_record = {}
buy_item_cancel = {}
buy_item = {}
cart = {}
games = {}
game_items = {}
game_game_type = {}
game_reviews = {}
publishers = {}
game_publishers = {}
user_games = {}
user_game_statistics = {}
user_inventory = {}
achievements = {}
user_achievements = {}

In [ ]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone

# Setup Faker for generating mock data
fake = faker.Faker()

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

def generate_users(num=10):
    for user_id in range(1, num + 1):  # Generate user IDs from 1 to num
        users[user_id] = {
            "user_id": user_id,
            "password_hashed": fake.password(length=20),
            "user_name": fake.user_name()[:10],
            "user_description": fake.text(max_nb_chars=300),
            "profile_pic": fake.image_url(),
            "profile_background": fake.image_url(),
            "birth_day": fake.date_of_birth(minimum_age=18, maximum_age=80),
            "email": fake.email(),
            "country": fake.country()[:20],
            "language": fake.language_name()[:20],
            "fund": random.randint(100, 1000),
            "filtering": random.choice([True, False]),
            "notification": random.choice([True, False]),
            "cookies": random.choice([True, False]),
        }

        # Use the correct dictionary values
        cur.execute(
            """
            INSERT INTO public."user" 
            (password_hashed, user_name, user_description, profile_pic, 
            profile_background, birthday, email, country, language, fund, 
            filtering, notification, cookies) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                users[user_id]['password_hashed'], 
                users[user_id]['user_name'], 
                users[user_id]['user_description'], 
                users[user_id]['profile_pic'], 
                users[user_id]['profile_background'], 
                users[user_id]['birth_day'], 
                users[user_id]['email'], 
                users[user_id]['country'], 
                users[user_id]['language'], 
                users[user_id]['fund'], 
                users[user_id]['filtering'], 
                users[user_id]['notification'], 
                users[user_id]['cookies']
            )
        )
    conn.commit()

# Call the function to generate 10,000 users with IDs from 1 to 10,000
generate_users(1000)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [7]:
# Define a function to generate user devices
def generate_user_devices(num=10):
    device_id_counter = 1  # Start device ID from 1
    for user_id in users.keys():  # Iterate over each user in the users dictionary
        num_devices = random.randint(1, 3)  # Generate a random number of devices for each user (e.g., 1 to 3 devices)
        for _ in range(num_devices):
            device_name = fake.word()[:20]  # Generate a device name
            type = fake.word()[:10]  # Generate a device type

            # Insert the device into the database
            cur.execute(
                """
                INSERT INTO public."user_devices" 
                (user_id, device_id, device_name, type) 
                VALUES (%s, %s, %s, %s)
                """,
                (user_id, device_id_counter, device_name, type)
            )
            device_id_counter += 1  # Increment the device ID after each insertion
    
    conn.commit()

# Call the function to generate user devices for all users
generate_user_devices(3)

ForeignKeyViolation: insert or update on table "user_devices" violates foreign key constraint "user_devices_user_id_fkey"
DETAIL:  Key (user_id)=(1) is not present in table "user".


In [98]:
# Define a function to generate user friends
def generate_user_friends(num=1):
    user_friends = {}  # Dictionary to store friendships
    
    for _ in range(num):
        user_id = random.choice(list(users.keys()))  # Pick a random user
        friend_id = random.choice(list(users.keys()))  # Pick a random friend

        # Ensure the friend_id is not the same as user_id
        while friend_id == user_id:
            friend_id = random.choice(list(users.keys()))
        
        # Add the friendship in both directions (user -> friend and friend -> user)
        if user_id not in user_friends:
            user_friends[user_id] = []
        if friend_id not in user_friends:
            user_friends[friend_id] = []

        # Add friend_id to user_id's friend list and vice versa
        user_friends[user_id].append(friend_id)
        user_friends[friend_id].append(user_id)

        # Insert the friendship into the database (both directions)
        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id) 
            VALUES (%s, %s)
            """,
            (user_id, friend_id)
        )

        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id) 
            VALUES (%s, %s)
            """,
            (friend_id, user_id)
        )
    
    conn.commit()

    # Print the friendships dictionary for verification
    print(user_friends)

# Call the function to generate and print user friends
generate_user_friends(5)

{280: [707], 707: [280], 891: [743], 743: [891], 393: [277], 277: [393], 937: [132], 132: [937], 808: [446], 446: [808]}


In [99]:
# Define a function to generate game types with sequential IDs
def generate_game_types(num=5):
    # Initialize game type ID to 1
    game_type_id = 1
    
    for _ in range(num):
        game_type_name = fake.word()[:10]  # Generate a random game type name
        
        # Store the generated data (optional, depending on your needs)
        game_types[game_type_id] = {
            "game_type_id": game_type_id,
            "game_type_name": game_type_name,
        }
        
        # Insert the game type into the database with sequential ID
        cur.execute(
            """
            INSERT INTO public."game_types" (game_type_id, game_type_name) 
            VALUES (%s, %s)
            """,
            (game_type_id, game_type_name)
        )
        
        # Increment the game type ID for the next game type
        game_type_id += 1
    
    conn.commit()

# Call the function to generate 20 game types with sequential IDs
generate_game_types(20)

In [100]:
def generate_user_game_types(num=5):
    for user_id in users.keys():  # Iterate over every user
        game_type_id = random.choice(list(game_types.keys()))  # Pick a random game type ID
        
        # Store the generated data (optional, depending on your needs)
        user_game_types[(user_id, game_type_id)] = {
            "user_id": user_id,
            "game_type_id": game_type_id,
        }
        
        # Insert the user-game type relationship into the database
        cur.execute(
            """
            INSERT INTO public."user_game_types" (user_id, game_type_id) 
            VALUES (%s, %s)
            """,
            (user_id, game_type_id)
        )
    conn.commit()

# Call the function to generate and insert game types for each user
generate_user_game_types(100)


In [101]:
def generate_add_fund_record(num=10):
    # Set a timezone (e.g., UTC) to convert the timestamp
    tz = timezone('UTC')
    
    for record_id in range(1, num + 1):  # Sequential IDs starting from 1
        user_id = random.choice(list(users.keys()))  # Select a random user
        fund_change = random.randint(10, 1000)  # Random fund change between 10 and 1000
        
        # Generate timestamp and convert to include time zone
        timestamp = fake.date_time_this_year()  # Random timestamp within this year
        timestamp = tz.localize(timestamp)  # Localize the timestamp to the UTC time zone
        
        # Insert the generated record into the add_fund_record table
        cur.execute(
            """
            INSERT INTO public."add_fund_record" (add_fund_record_id, user_id, fund_change, timestamp)
            VALUES (%s, %s, %s, %s)
            """,
            (record_id, user_id, fund_change, timestamp)
        )
    conn.commit()

# Call the function to generate and insert 500 add fund records with sequential IDs
generate_add_fund_record(500)

In [102]:
# Define a function to generate games
def generate_games(num=10):
    for game_id in range(1, num + 1):  # Sequential IDs starting from 1
                
        # Generate random game attributes
        game_name = fake.word()[:20]
        game_description = fake.sentence()[:100]
        system_requirements = fake.sentence()[:100]
        
        # Store the generated game record in the dictionary (optional)
        games[game_id] = {
            "game_id": game_id,
            "game_name": game_name,
            "game_description": game_description,
            "system_requirements": system_requirements,
        }
        
        # Insert the generated record into the database
        cur.execute(
            """
            INSERT INTO public."game" (game_id, game_name, game_description, system_requirements)
            VALUES (%s, %s, %s, %s)
            """,
            (game_id, game_name, game_description, system_requirements)
        )
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 game records
generate_games(2000)


In [103]:
def generate_game_items(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    # Track the current item_id for each game_id
    item_id_counter = {game_id: 0 for game_id in games.keys()}  # Initialize counters for each game_id

    for game_id in games.keys():  # Loop through each existing game ID
        for _ in range(num):
            # Generate sequential item_id for each game_id
            item_id = item_id_counter[game_id] + 1
            item_id_counter[game_id] = item_id  # Increment the counter for the next item_id
            
            # Generate random attributes for the game item
            original_price = random.randint(10, 100)
            current_price = random.randint(10, 100)
            special_offer = round(random.uniform(0, 1), 2)
            release_date = fake.date_this_decade()
            
            # Store the generated game item in the dictionary using a tuple (item_id, game_id) as the key
            game_items[(item_id, game_id)] = {
                "item_id": item_id,
                "game_id": game_id,
                "original_price": original_price,
                "current_price": current_price,
                "special_offer": special_offer,
                "release_date": release_date,
            }
            
            # Insert the generated game item into the database
            cur.execute(
                """
                INSERT INTO public."game_item" (item_id, game_id, original_price, current_price, special_offer, release_date) 
                VALUES (%s, %s, %s, %s, %s, %s)
                """,
                (item_id, game_id, original_price, current_price, special_offer, release_date)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 100 game items with sequential item IDs for each game_id
generate_game_items(10)


In [104]:
def generate_buy_item(num=10):
    tz = timezone('UTC')  # Set the timezone (e.g., UTC)
    
    for user_id in users.keys():  # Loop over each user
        for _ in range(num):  # Generate 10 buy items for each user
            # Generate a unique buy_item_id
            buy_item_id = fake.unique.random_number(digits=5)
            
            # Pick a (game_id, item_id) tuple from the existing records
            item_id, game_id = random.choice(list(game_items.keys()))
            
            # Random price between 10 and 100
            price = random.randint(10, 100)
            
            # Generate timestamp and localize it to UTC timezone
            timestamp = fake.date_time_this_year()
            timestamp = tz.localize(timestamp)  # Localize to the UTC time zone
            
            # Randomly choose whether the buy item is cancelled
            isCancelled = random.choice([True, False])
            
            # Store the generated buy item record in the dictionary (optional)
            buy_item[(buy_item_id, game_id, item_id)] = {
                "buy_item_id": buy_item_id,
                "user_id": user_id,
                "game_id": game_id,
                "item_id": item_id,
                "price": price,
                "timestamp": timestamp,
                "isCancelled": isCancelled
            }
            
            # Insert the generated record into the database
            cur.execute(
                """
                INSERT INTO public."buy_item" (buy_item_id, user_id, game_id, item_id, price, timestamp, "isCancelled")
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (buy_item_id, user_id, game_id, item_id, price, timestamp, isCancelled)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 buy item records for each user
generate_buy_item(10)

In [105]:
# Define a function to generate buy item cancel records
def generate_buy_item_cancel(num=10):
    if not buy_item:
        raise ValueError("Buy item records have not been generated yet.")
    
    tz = timezone('UTC')  # Set the timezone (e.g., UTC)
    
    # Filter buy items with isCancelled = True
    cancelled_buy_items = {k: v for k, v in buy_item.items() if v["isCancelled"] is True}
    
    if not cancelled_buy_items:
        raise ValueError("No cancelled buy items found.")

    for _ in range(num):
        # Pick a random buy_item_id, game_id, and item_id from the cancelled items
        buy_item_id, game_id, item_id = random.choice(list(cancelled_buy_items.keys()))
        
        # Generate a cancellation timestamp and localize it to UTC timezone
        timestamp = fake.date_time_this_year()
        timestamp = tz.localize(timestamp)  # Localize to UTC time zone
        
        # Store the generated cancel record in the dictionary (optional)
        buy_item_cancel[(buy_item_id, game_id, item_id)] = {
            "buy_item_id": buy_item_id,
            "game_id": game_id,
            "item_id": item_id,
            "timestamp": timestamp
        }
        
        # Insert the cancel record into the database
        cur.execute(
            """
            INSERT INTO public."buy_item_cancel" (buy_item_id, game_id, item_id, timestamp)
            VALUES (%s, %s, %s, %s)
            """,
            (buy_item_id, game_id, item_id, timestamp)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 5 buy item cancel records
generate_buy_item_cancel(100)

In [106]:
def generate_cart(num=10):
    if not games or not game_items:
        raise ValueError("Games or game items have not been generated yet.")
    
    for user_id in users.keys():  # Iterate over all users
        for _ in range(num):  # Generate 10 cart items for each user
            # Pick a (game_id, item_id) tuple from the existing records
            item_id, game_id = random.choice(list(game_items.keys()))
            
            # Store the generated cart item in the dictionary (optional)
            cart[(user_id, game_id, item_id)] = {
                "user_id": user_id,
                "game_id": game_id,
                "item_id": item_id
            }
            
            # Insert the cart item into the database
            cur.execute(
                """
                INSERT INTO public."cart" (user_id, game_id, item_id)
                VALUES (%s, %s, %s)
                """,
                (user_id, game_id, item_id)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 cart items for each user
generate_cart(3)

In [107]:
def generate_game_game_type():
    if not games or not game_types:
        raise ValueError("Games or game types have not been generated yet.")
    
    for game_id in games.keys():  # Iterate over each game
        # Randomly choose how many game types to associate with the game (between 1 and 3)
        num_game_types = random.randint(1, 3)
        
        # Select random game types for the current game
        selected_game_types = random.sample(list(game_types.keys()), num_game_types)
        
        for game_type_id in selected_game_types:
            # Store the generated game-game type pair (optional)
            game_game_type[(game_id, game_type_id)] = {
                "game_id": game_id,
                "game_type_id": game_type_id
            }
            
            # Insert the generated pair into the database
            cur.execute(
                """
                INSERT INTO public."game_game_type" (game_id, game_type_id)
                VALUES (%s, %s)
                """,
                (game_id, game_type_id)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert game-game type records
generate_game_game_type()

In [108]:
def generate_game_reviews():
    if not games or not users:
        raise ValueError("Games or users have not been generated yet.")
    
    for game_id in games.keys():  # Iterate over each game
        # Randomly choose how many reviews to associate with the game (between 1 and 3)
        num_reviews = random.randint(1, 3)
        
        for _ in range(num_reviews):
            # Generate a unique review_id
            review_id = fake.unique.random_number(digits=9)
            
            # Pick a random user_id for the review
            user_id = random.choice(list(users.keys()))
            
            # Generate review attributes
            review_timestamp = fake.date_time_this_year()
            rating = random.randint(1, 5)
            text = fake.sentence()
            
            # Store the generated review record (optional)
            game_reviews[review_id] = {
                "review_id": review_id,
                "game_id": game_id,
                "user_id": user_id,
                "review_timestamp": review_timestamp,
                "rating": rating,
                "text": text
            }
            
            # Insert the review record into the database
            cur.execute(
                """
                INSERT INTO public."game_reviews" (review_id, game_id, user_id, review_timestamp, rating, text)
                VALUES (%s, %s, %s, %s, %s, %s)
                """,
                (review_id, game_id, user_id, review_timestamp, rating, text)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert game reviews
generate_game_reviews()

In [109]:
def generate_publishers(num=10):
    for publisher_id in range(1, num + 1):  # Start from 1 and increment by 1
        # Generate publisher attributes
        publisher_name = fake.company()[:20]
        description = fake.sentence()[:100]
        
        # Store the generated publisher record (optional)
        publishers[publisher_id] = {
            "publisher_id": publisher_id,
            "publisher_name": publisher_name,
            "description": description,
        }
        
        # Insert the publisher data into the database
        cur.execute(
            """
            INSERT INTO public."publishers" (publisher_id, publisher_name, description)
            VALUES (%s, %s, %s)
            """,
            (publisher_id, publisher_name, description)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 100 publisher records
generate_publishers(100)

In [113]:
def generate_game_publishers(num=10):
    if not publishers or not games:
        raise ValueError("publishers or games have not been generated yet.")
    
    for game_id in games.keys():
        # Select 1 or 2 publishers for each game
        num_publishers = random.randint(1, 2)  # Either 1 or 2 publishers
        
        # Ensure each game gets 1 to 2 publishers
        assigned_publishers = random.sample(list(publishers.keys()), num_publishers)
        
        for publisher_id in assigned_publishers:
            # Store the game-publisher relationship in the dictionary (optional)
            game_publishers[(publisher_id, game_id)] = {
                "publisher_id": publisher_id,
                "game_id": game_id
            }
            
            # Insert the game-publisher relationship into the database
            cur.execute(
                """
                INSERT INTO public."game_publishers" (publisher_id, game_id)
                VALUES (%s, %s)
                """,
                (publisher_id, game_id)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert game-publisher relationships
generate_game_publishers(100)

In [114]:
def generate_user_games(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    for user_id in users.keys():  # Loop through each user
        # Randomly select between 1 and 20 games for each user
        num_games = random.randint(1, 20)  # Between 1 and 20 games per user
        
        # Select the games for this user
        selected_games = random.sample(list(games.keys()), num_games)
        
        for game_id in selected_games:
            # Generate installation date
            installed_date = fake.date_this_decade()
            
            # Generate uninstallation date, or set it to None if not uninstalled
            uninstalled_date = fake.date_this_decade() if random.choice([True, False]) else None
            
            # Store the user-game relationship in the dictionary (optional)
            user_games[(user_id, game_id)] = {
                "user_id": user_id,
                "game_id": game_id,
                "installed_date": installed_date,
                "uninstalled_date": uninstalled_date
            }
            
            # Insert the user-game relationship into the database
            cur.execute(
                """
                INSERT INTO public."user_games" (user_id, game_id, installed_date, uninstalled_date)
                VALUES (%s, %s, %s, %s)
                """,
                (user_id, game_id, installed_date, uninstalled_date)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert user-game relationships
generate_user_games(1000)

In [115]:
def generate_user_game_statistics():
    if not user_games:  # Ensure that user_games are generated
        raise ValueError("User games have not been generated yet.")
    
    for user_id, game_id in user_games.keys():  # Loop through each user-game pair
        # Generate random statistics for played time (in minutes) and number of achievements
        played_time = timedelta(minutes=random.randint(1, 1000))
        achievement_num = random.randint(0, 50)
        
        # Store the user game statistics in the dictionary (optional)
        user_game_statistics[(user_id, game_id)] = {
            "user_id": user_id,
            "game_id": game_id,
            "played_time": played_time,
            "achievement_num": achievement_num
        }
        
        # Insert the generated user game statistics into the database
        cur.execute(
            """
            INSERT INTO public."user_game_statistics" (user_id, game_id, played_time, achievement_num)
            VALUES (%s, %s, %s, %s)
            """,
            (user_id, game_id, played_time, achievement_num)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert user game statistics
generate_user_game_statistics()

In [116]:
def generate_user_inventory():
    if not games or not game_items or not user_games:
        raise ValueError("Games, game items, or user games have not been generated yet.")
    
    # Define a timezone (you can replace 'UTC' with any other timezone as needed)
    tz = pytz.timezone('UTC')

    # Loop through each user
    for user_id, game_id in user_games.keys():
        # Determine how many items to generate for this user (between 1 and 3)
        num_items = random.randint(1, 3)
        
        # Filter the game_items to those that belong to the selected game_id
        available_items = [item_id for item_id, item in game_items.items() if item['game_id'] == game_id]
        
        if not available_items:
            continue  # Skip if no items are available for the selected game_id
        
        # Pick random items from the available items for the selected game_id
        selected_items = random.sample(available_items, min(num_items, len(available_items)))  # Ensure no duplicates
        
        for item_id in selected_items:
            # Ensure the (game_id, item_id) pair exists in the game_items table
            if (game_id, item_id) not in game_items:
                continue  # Skip if the item_id does not exist for the current game_id in the game_items table
            
            # Generate random dates for acquisition and "not owned" status
            acquired_date = fake.date_this_century()  # Date when the item was acquired
            
            # Convert acquired_date (a datetime.date object) to datetime and localize
            acquired_datetime = datetime.combine(acquired_date, datetime.min.time())  # Add time part to make it a datetime
            acquired_datetime = tz.localize(acquired_datetime)  # Make acquired_date timezone-aware
            
            # Generate "not owned" date randomly, it can be None or a valid date
            not_owned_date = fake.date_this_century() if random.choice([True, False]) else None
            if not_owned_date:
                # Convert not_owned_date (a datetime.date object) to datetime and localize
                not_owned_datetime = datetime.combine(not_owned_date, datetime.min.time())  # Add time part
                not_owned_datetime = tz.localize(not_owned_datetime)  # Make not_owned_date timezone-aware
            
            # If not_owned_date is set, ensure it happens after the acquired_datetime
            if not_owned_date and not_owned_datetime < acquired_datetime:
                not_owned_datetime = fake.date_between(start_date=acquired_datetime)  # Correct to use `date_between`
                # Convert new not_owned_date to datetime and localize
                not_owned_datetime = datetime.combine(not_owned_datetime, datetime.min.time())
                not_owned_datetime = tz.localize(not_owned_datetime)
            
            # Store the user inventory record
            user_inventory[(user_id, game_id, item_id)] = {
                "user_id": user_id,  # No need to convert to string if user_id is an integer
                "game_id": game_id,  # No need to convert to string if game_id is an integer
                "item_id": item_id,  # No need to convert to string if item_id is an integer
                "acquired_date": acquired_datetime,
                "not_owned_date": not_owned_datetime
            }
            
            # Debugging: Check the inventory data before inserting
            print(f"Inserting user inventory for user {user_id}, game {game_id}, item {item_id}")
            print(f"SQL: INSERT INTO public.\"user_inventory\" (user_id, game_id, item_id, acquired_date, not_owned_date) "
                 f"VALUES ({user_id}, {game_id}, {item_id}, '{acquired_datetime}', '{not_owned_datetime}')")
            
            # Insert the user inventory record into the database
            cur.execute(
                """
                INSERT INTO public."user_inventory" (user_id, game_id, item_id, acquired_date, not_owned_date)
                VALUES (%s, %s, %s, %s, %s)
                """,
                (user_id, game_id, item_id, acquired_datetime, not_owned_datetime)
            )
    
    # Commit the changes to the database
    conn.commit()
generate_user_inventory()

KeyboardInterrupt: 

In [117]:
def generate_achievements(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    # For each game, we will start achievement_id from 1
    for game_id in games.keys():
        # Number of achievements for the current game (between 1 and 5)
        num_achievements = random.randint(1, 5)
        
        # Start the achievement_id from 1 for each game
        achievement_id_counter = 1
        
        for _ in range(num_achievements):
            # Use the counter for achievement_id to ensure it starts from 1 and increments sequentially per game
            achievement_id = achievement_id_counter
            achievement_id_counter += 1
            
            # Generate random attributes for the achievement
            achievement_name = fake.word()[:10]
            achievement_description = fake.sentence()[:100]
            
            # Store the generated achievement in the dictionary
            achievements[(achievement_id, game_id)] = {
                "achievement_id": achievement_id,
                "game_id": game_id,
                "achievement_name": achievement_name,
                "achievement_description": achievement_description,
            }
            
            # Insert the generated achievement into the database
            cur.execute(
                """
                INSERT INTO public."achievements" (achievement_id, game_id, achievement_name, achievement_description) 
                VALUES (%s, %s, %s, %s)
                """,
                (achievement_id, game_id, achievement_name, achievement_description)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert achievements, ensuring 1 to 5 per game
generate_achievements(len(games) * 3)  # Adjust the total number based on your needs

In [87]:
def generate_user_achievements():
    if not achievements:
        raise ValueError("Achievements have not been generated yet.")
    
    tz = pytz.timezone('UTC')  # Set your desired timezone (e.g., 'UTC')
    
    for user_id, game_id in user_games.keys():
        # Determine how many achievements to assign to this user-game pair (between 1 and 3)
        num_achievements = random.randint(1, 3)
        
        # Get available achievements for the selected game_id
        available_achievements = [ach_id for ach_id, ach in achievements.items() if ach['game_id'] == game_id]

        if not available_achievements:
            print(f"No achievements available for game_id {game_id}")  # Debugging line
            continue  # Skip if no achievements are available for the selected game_id
        
        # Pick a random subset of achievements (1 to 3)
        selected_achievements = random.sample(available_achievements, min(num_achievements, len(available_achievements)))
        

        for achievement_id in selected_achievements:
            # Ensure the achievement_id and game_id pair exists in the achievements table
            if (achievement_id, game_id) not in achievements:
                continue  # Skip this entry if the combination is invalid
            
            # Generate a random date for when the achievement was achieved
            achieved_date = fake.date_this_century()
            
            # Convert the date to a datetime object and make it timezone-aware
            achieved_date = datetime.combine(achieved_date, datetime.min.time())  # Convert to datetime
            achieved_date = tz.localize(achieved_date)  # Make it timezone-aware
            
            # Store the generated user achievement in the dictionary
            user_achievements[(user_id, game_id, achievement_id)] = {
                "user_id": user_id,
                "game_id": game_id,
                "achievement_id": achievement_id,
                "achieved_date": achieved_date
            }
            
            # Insert the generated user achievement into the database
            cur.execute(
                """
                INSERT INTO public."user_achievements" (user_id, game_id, achievement_id, achieved_date)
                VALUES (%s, %s, %s, %s)
                """,
                (user_id, game_id, achievement_id, achieved_date)
            )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert user achievements
generate_user_achievements()

In [2]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone

# Setup Faker for generating mock data
fake = faker.Faker()

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

# List of SQL delete statements
sql_delete_queries = [
    "DELETE FROM public.\"user_achievements\";",
    "DELETE FROM public.\"user_game_statistics\";",
    "DELETE FROM public.\"user_games\";",
    "DELETE FROM public.\"game_publishers\";",
    "DELETE FROM public.\"publishers\";",
    "DELETE FROM public.\"game_reviews\";",
    "DELETE FROM public.\"game_game_type\";",

    "DELETE FROM public.\"cart\";",
    "DELETE FROM public.\"buy_item_cancel\";",
    "DELETE FROM public.\"buy_item\";",
    "DELETE FROM public.\"add_fund_record\";",
    "DELETE FROM public.\"user_friends\";",
    "DELETE FROM public.\"user_devices\";",
    "DELETE FROM public.\"user_game_types\";",
    "DELETE FROM public.\"achievements\";",
    "DELETE FROM public.\"game_item\";",
    "DELETE FROM public.\"game\";",
    "DELETE FROM public.\"user\";",
    "DELETE FROM public.\"game_types\";"
]

# Execute each SQL query
for query in sql_delete_queries:
    cur.execute(query)

# Commit the changes to the database
conn.commit()